In [2]:
import numpy as np
import pywt
import os
from PIL import Image
from scipy.fftpack import dct
from scipy.fftpack import idct
import cv2

### Implementing the watermarking algorithm for video

#### Wavelet Transform

In [ ]:
def wavelet_tf(imArray, model, level):
    coeffs=pywt.wavedec2(data = imArray, wavelet = model, level = level)
    # print coeffs[0].__len__()
    coeffs_H=list(coeffs) 
   
    return coeffs_H


#### Embed Watermark

In [ ]:
def embed_watermark(watermark_array, orig_image):
    watermark_array_size = watermark_array[0].__len__() # Gets the size of the watermark array
    watermark_flat = watermark_array.ravel() # Flattens the watermark array
    ind = 0

    for x in range (0, orig_image.__len__(), 8): # Loops through the image in 8x8 blocks
        for y in range (0, orig_image.__len__(), 8):
            if ind < watermark_flat.__len__(): # While in the watermark array size
                subdct = orig_image[x:x+8, y:y+8] # Gets the 8x8 block of the image
                subdct[5][5] = watermark_flat[ind] # Sets the 5,5 value of the block to the watermark value
                orig_image[x:x+8, y:y+8] = subdct # Sets the 8x8 block of the image to the new block
                ind += 1 # Next Watermark's pixel value


    return orig_image

Why 8x8 blocks ? Maybe so that the compression (by block) alters less of the watermark.

#### DCT Transform

In [ ]:
def apply_dct(image_array): # Applies the DCT to each 8x8 block of the image
    size = image_array[0].__len__()
    all_subdct = np.empty((size, size))
    for i in range (0, size, 8):
        for j in range (0, size, 8):
            subpixels = image_array[i:i+8, j:j+8]
            subdct = dct(dct(subpixels.T, norm="ortho").T, norm="ortho")
            all_subdct[i:i+8, j:j+8] = subdct

    return all_subdct

In [ ]:
def inverse_dct(all_subdct):
    size = all_subdct[0].__len__()
    all_subidct = np.empty((size, size))
    for i in range (0, size, 8):
        for j in range (0, size, 8):
            subidct = idct(idct(all_subdct[i:i+8, j:j+8].T, norm="ortho").T, norm="ortho")
            all_subidct[i:i+8, j:j+8] = subidct

    return all_subidct

#### Retrieve the watermark

In [ ]:
def get_watermark(dct_watermarked_coeff, watermark_size): 
    # Gets the [5,5] coefficient of each 8x8 block of the watermarked image and puts it into a 1D array
    
    subwatermarks = []

    for x in range (0, dct_watermarked_coeff.__len__(), 8):
        for y in range (0, dct_watermarked_coeff.__len__(), 8):
            coeff_slice = dct_watermarked_coeff[x:x+8, y:y+8]
            subwatermarks.append(coeff_slice[5][5])

    watermark = np.array(subwatermarks).reshape(watermark_size, watermark_size)
    #Reshapes the flattened array into a 2D array
    return watermark

In [ ]:
def recover_watermark(image_array, model='haar', level = 1):


    coeffs_watermarked_image = process_coefficients(image_array, model, level=level)
    dct_watermarked_coeff = apply_dct(coeffs_watermarked_image[0])
    
    watermark_array = get_watermark(dct_watermarked_coeff, 128)

    watermark_array =  np.uint8(watermark_array)


### Running the algorithm on a video

In [ ]:
watermark = 

In [ ]:
def watermark_image(img):
    model = 'haar'
    level = 1
    image_array = convert_image(image, 2048)
    watermark_array = convert_image(watermark, 128)

    coeffs_image = process_coefficients(image_array, model, level=level)
    dct_array = apply_dct(coeffs_image[0])
    dct_array = embed_watermark(watermark_array, dct_array)
    coeffs_image[0] = inverse_dct(dct_array)
  
    image_array_H=pywt.waverec2(coeffs_image, model)

    recover_watermark(image_array = image_array_H, model=model, level = level)

In [ ]:
cap = cv2.VideoCapture('oppenheimer_trailer.mp4')
total_frames = last_frame_number = cap.get(cv2.CAP_PROP_FRAME_COUNT)

frame_number = -1
while True:
    frame_number += 1
    cap.set(1, frame_number)
    ret, frame = cap.read()
    if not ret or frame_number >= last_frame_number:
        break

    # Modify the pixels of the frame here
    # For example, convert the frame to grayscale
    watermarked_image = watermark_image(frame)

    # Write the modified frame to a new video file
    out.write(gray_frame)

cap.release()
out.release()
